In [28]:
import config

import logging
import random
import numpy as np
import matplotlib.pyplot as plt

logger = logging.getLogger(__name__)

In [17]:
import glob
import pickle

files = glob.glob('embeddings/*_test.pkl')
embeddings_data = []

for file in files:
    logger.info(f'Loading {file}')
    with open(file, 'rb') as fp:
        dict = pickle.load(fp)
        embeddings_data.append(dict)

2023-04-20 18:17:01,786 - INFO     | __main__   | Loading embeddings/glove_embeddings_test.pkl
2023-04-20 18:17:01,788 - INFO     | __main__   | Loading embeddings/mpnet_embeddings_test.pkl
2023-04-20 18:17:01,789 - INFO     | __main__   | Loading embeddings/distil_embeddings_test.pkl
2023-04-20 18:17:01,790 - INFO     | __main__   | Loading embeddings/wiki_embeddings_test.pkl
2023-04-20 18:17:01,791 - INFO     | __main__   | Loading embeddings/w2v_embeddings_test.pkl


# Clustering

Testing on mpnet embeddings of Llama 7B output.

In [22]:
mpnet_llama = embeddings_data[1]['llama_7b']

In [24]:
# plotting
def plot_clusters(embeddings: list, clusters, n_neighbors, min_dist: float):
    umap_data = umap.UMAP(n_neighbors=n_neighbors, 
                          n_components=2, 
                          min_dist = min_dist,  
                          #metric='cosine',
                          random_state=42).fit_transform(embeddings)
    
    point_size = 100.0 / np.sqrt(embeddings.shape[0])
    
    result = pd.DataFrame(umap_data, columns=['x', 'y'])
    result['labels'] = clusters.labels_

    fig, ax = plt.subplots(figsize=(14, 8))
    outliers = result[result.labels == -1]
    clustered = result[result.labels != -1]
    plt.scatter(outliers.x, outliers.y, color = 'lightgrey', s=point_size)
    plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=point_size, cmap='jet')
    plt.colorbar()
    plt.show()

## UMAP + HDBSCAN

In [11]:
import umap
import hdbscan